In [5]:
import kagglehub

path = kagglehub.dataset_download("roblexnana/the-babi-tasks-for-nlp-qa-system")

print("Path to dataset files:", path)

100%|█████████████████████████████████████████████████████████████████████████████| 16.7M/16.7M [00:02<00:00, 5.83MB/s]

Extracting files...


Path to dataset files: C:\Users\ADMIN\.cache\kagglehub\datasets\roblexnana\the-babi-tasks-for-nlp-qa-system\versions\1


In [7]:
import os
import re

def load_babi_data(path):
    stories, questions, answers = [], [], []
    story = []

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            idx, text = line.split(" ", 1)

            if idx == "1":
                story = []

            if "?" in text:
                q, a, _ = text.split("\t")
                stories.append(" ".join(story))
                questions.append(q.replace("?", ""))
                answers.append(a)
            else:
                story.append(text)

    return stories, questions, answers

In [9]:
train_path = r"C:\Users\ADMIN\.cache\kagglehub\datasets\roblexnana\the-babi-tasks-for-nlp-qa-system\versions\1\tasks_1-20_v1-2\en\qa1_single-supporting-fact_train.txt"
test_path = r"C:\Users\ADMIN\.cache\kagglehub\datasets\roblexnana\the-babi-tasks-for-nlp-qa-system\versions\1\tasks_1-20_v1-2\en\qa1_single-supporting-fact_test.txt"

train_stories, train_questions, train_answers = load_babi_data(train_path)
test_stories, test_questions, test_answers = load_babi_data(test_path)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(train_stories + train_questions)

vocab_size = len(tokenizer.word_index) + 1

max_story_len = max(len(s.split()) for s in train_stories)
max_question_len = max(len(q.split()) for q in train_questions)

def vectorize(stories, questions):
    story_seq = pad_sequences(
        tokenizer.texts_to_sequences(stories),
        maxlen=max_story_len
    )
    question_seq = pad_sequences(
        tokenizer.texts_to_sequences(questions),
        maxlen=max_question_len
    )
    return story_seq, question_seq

In [27]:
X_story_train, X_q_train = vectorize(train_stories, train_questions)
X_story_test, X_q_test = vectorize(test_stories, test_questions)
print(train_stories)
print(train_questions)
print(train_answers)

['Mary moved to the bathroom. John went to the hallway.', 'Mary moved to the bathroom. John went to the hallway. Daniel went back to the hallway. Sandra moved to the garden.', 'Mary moved to the bathroom. John went to the hallway. Daniel went back to the hallway. Sandra moved to the garden. John moved to the office. Sandra journeyed to the bathroom.', 'Mary moved to the bathroom. John went to the hallway. Daniel went back to the hallway. Sandra moved to the garden. John moved to the office. Sandra journeyed to the bathroom. Mary moved to the hallway. Daniel travelled to the office.', 'Mary moved to the bathroom. John went to the hallway. Daniel went back to the hallway. Sandra moved to the garden. John moved to the office. Sandra journeyed to the bathroom. Mary moved to the hallway. Daniel travelled to the office. John went back to the garden. John moved to the bedroom.', 'Sandra travelled to the office. Sandra went to the bathroom.', 'Sandra travelled to the office. Sandra went to the

In [16]:
answer_tokenizer = Tokenizer(filters="")
answer_tokenizer.fit_on_texts(train_answers)

y_train = answer_tokenizer.texts_to_sequences(train_answers)
y_test = answer_tokenizer.texts_to_sequences(test_answers)

y_train = [a[0] for a in y_train]
y_test = [a[0] for a in y_test]

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model

embedding_dim = 64
num_heads = 4
ff_dim = 128

story_input = Input(shape=(max_story_len,))
question_input = Input(shape=(max_question_len,))

embedding = Embedding(vocab_size, embedding_dim)

story_embed = embedding(story_input)
question_embed = embedding(question_input)

attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
attn_output = attention(
    query=question_embed,
    value=story_embed,
    key=story_embed
)

x = LayerNormalization()(attn_output)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation="relu")(x)
output = Dense(len(answer_tokenizer.word_index) + 1, activation="softmax")(x)

model = Model(inputs=[story_input, question_input], outputs=output)

In [20]:
import numpy as np

y_train = np.array(y_train)
y_test = np.array(y_test)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    [X_story_train, X_q_train],
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - accuracy: 0.1603 - loss: 1.8675 - val_accuracy: 0.1100 - val_loss: 1.8535
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.1869 - loss: 1.8195 - val_accuracy: 0.1300 - val_loss: 1.8181
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.2005 - loss: 1.7836 - val_accuracy: 0.2200 - val_loss: 1.7228
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.2489 - loss: 1.7328 - val_accuracy: 0.1900 - val_loss: 1.7131
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.2802 - loss: 1.6780 - val_accuracy: 0.2300 - val_loss: 1.6639
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3381 - loss: 1.6035 - val_accuracy: 0.3900 - val_loss: 1.5586
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3770 - loss: 1.5054 - val_accuracy: 0.3200 - val_loss: 1.7214
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3707 - loss: 1.5564 - val_accuracy: 0.4100 - v

In [21]:
loss, acc = model.evaluate([X_story_test, X_q_test], y_test)
print("Transformer QA Accuracy:", acc)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4110 - loss: 1.3403
Transformer QA Accuracy: 0.4129999876022339


In [22]:
from tensorflow.keras.layers import LSTM, Concatenate
story_lstm = LSTM(64)(story_embed)
question_lstm = LSTM(64)(question_embed)

merged = Concatenate()([story_lstm, question_lstm])
dense = Dense(64, activation="relu")(merged)
lstm_output = Dense(len(answer_tokenizer.word_index) + 1, activation="softmax")(dense)

lstm_model = Model([story_input, question_input], lstm_output)

In [23]:
lstm_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

lstm_model.fit(
    [X_story_train, X_q_train],
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1
)

lstm_loss, lstm_acc = lstm_model.evaluate([X_story_test, X_q_test], y_test)
print("LSTM QA Accuracy:", lstm_acc)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 102ms/step - accuracy: 0.2922 - loss: 1.8827 - val_accuracy: 0.4400 - val_loss: 1.6765
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.4523 - loss: 1.6054 - val_accuracy: 0.5600 - val_loss: 1.4982
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.5593 - loss: 1.3754 - val_accuracy: 0.5100 - val_loss: 1.2945
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 0.5483 - loss: 1.2575 - val_accuracy: 0.5200 - val_loss: 1.3931
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 0.6132 - loss: 1.0954 - val_accuracy: 0.5300 - val_loss: 1.2714
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.5797 - loss: 1.1234 - val_accuracy: 0.5600 - val_loss: 1.2062
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.6047 - loss: 1.0650 - val_accuracy: 0.5000 - val_loss: 1.2453
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.6020 - loss: 1.0503 - val_accuracy: 0.5000 -